In [ ]:
import h5py
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.metrics import roc_curve, auc
import tensorflow.keras.backend as K
import qkeras
from qkeras import *

np.random.seed(42)
tf.random.set_seed(42)

# truncate input with max number of constituents per jet
N = 16

with h5py.File('data.h5', 'r') as f:
    x = f['x'][:, :N, :]
    y = f['y'][:]

# min pt cut on jet constituents
x[x[:, :, 0] < 2] = 0

non_zero_counts = np.sum(np.any(x == 0, axis=(2)), axis=1)
plt.figure(figsize=(8, 6))
plt.hist(non_zero_counts, bins=range(0, N+2), edgecolor='black', alpha=0.5)
plt.xlabel(f"Number of zero-padded constituents per jet (max {N} constituents considered per jet)")
plt.ylabel("Count of jets")
plt.yscale("log")
plt.show()

# normalization of the pt feature by interquantile range
q5 = np.percentile(x[:, :, 0], 5)
q95 = np.percentile(x[:, :, 0], 95)
x[:, :, 0] = (x[:, :, 0] - 0) / (q95 - q5)

train_ratio = 0.5
val_ratio = 0.1
test_ratio = 1 - train_ratio - val_ratio
X_train_val, X_test, Y_train_val, Y_test = train_test_split(x, y, test_size = test_ratio, random_state = 42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = val_ratio/(val_ratio + train_ratio), random_state = 42)
print('X_train shape: ' + str(X_train.shape))
print('X_val   shape: ' + str(X_val.shape))
print('X_test  shape: ' + str(X_test.shape))
print('Y_train shape: ' + str(Y_train.shape))
print('Y_val   shape: ' + str(Y_val.shape))
print('Y_test  shape: ' + str(Y_test.shape))
del X_train_val, Y_train_val

In [ ]:
phi_dim = 32
rho_dim = 32

quantizer = quantized_bits(8, 0, alpha=1)
quantized_relu = 'quantized_relu(8, 0)'

input_constituent = keras.Input(shape=(N, 3), name='phi_input')

'''
# phi network (dense)
x = QDense(phi_dim, use_bias=True, name='phi1', kernel_quantizer=quantizer, bias_quantizer=quantizer)(input_constituent)
x = QActivation(quantized_relu, name='relu1')(x)
    
x = QDense(phi_dim, use_bias=True, name='phi2', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu2')(x)
    
x = QDense(phi_dim, use_bias=True, name='phi3', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu3')(x)
'''

# phi network (pointwise conv1d)
x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi1',
            kernel_quantizer=quantizer, bias_quantizer=quantizer)(input_constituent)
x = QActivation(quantized_relu, name='relu1')(x)
    
x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi2',
            kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu2')(x)
    
x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi3',
            kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu3')(x)

# permutation-invariant Aggregation
x = QActivation(quantized_bits(20, 10, symmetric=0, keep_negative=1), name='act_agg')(x)
x = layers.GlobalAveragePooling1D(name='agg')(x)

# rho network
x = QDense(rho_dim, use_bias=True, name='rho1', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = QActivation(quantized_relu, name='relu4')(x)

x = QDense(5, use_bias=True, name='rho2', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
x = layers.Softmax(name='softmax')(x)

#model = keras.Model(input_constituent, x, name='deepset_densePhi')
model = keras.Model(input_constituent, x, name='deepset_pointwisePhi')

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.005),
              loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train,
                    validation_data = (X_val, Y_val),
                    epochs=10, batch_size=128)

In [ ]:
plt.figure(figsize = (15,10))
axes = plt.subplot(2, 2, 1)
axes.plot(history.history['loss'], label = 'train loss')
axes.plot(history.history['val_loss'], label = 'val loss')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
Y_pred = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))))

In [ ]:
def plot_roc(y_test, y_pred, labels):
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        plt.plot(fpr, tpr, label='{0} tagger, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-')
    #plt.semilogy()
    #plt.semilogx()
    plt.ylabel("Signal Efficiency")
    plt.xlabel("Background Efficiency")
    #plt.ylim(0.00001, 1)
    #plt.xlim(0.00001, 1)
    plt.grid(True)
    plt.legend(loc='best')  
    
plt.figure(figsize=(4, 4))
plot_roc(Y_test, Y_pred, ['g','q','w','z','t'])

In [ ]:
model.save('model-pointwisePhi.keras')
#model.save('model-densePhi.h5')

In [ ]:
model = tf.keras.models.load_model('model-pointwisePhi.keras')
#model = tf.keras.models.load_model('model-densePhi.h5')

In [ ]:
Y_pred = model.predict(X_test)

# HLS

In [ ]:
import hls4ml
model = tf.keras.models.Sequential()
model.add(Dense(64, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform'))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform'))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform'))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(5, name='output', kernel_initializer='lecun_uniform'))
model.add(Activation(activation='softmax', name='softmax'))
adam = tf.keras.optimizers.Adam(0.0001)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

In [ ]:
import hls4ml
#model = tf.keras.models.load_model('model-pointwisePhi.keras')
#model = tf.keras.models.load_model('model-densePhi.keras')

config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model']['Strategy'] = 'Latency'
config['LayerName']['phi_input']['Precision'] = 'ap_fixed<12, 4, AP_RND, AP_SAT>'

# for dense Phi
if False:
    config['LayerName']['phi1']['ParallelizationFactor'] = 1
    config['LayerName']['phi1']['ReuseFactor'] = 8
    config['LayerName']['phi1']['Strategy'] = 'Latency'
    #config['LayerName']['phi1']['ConvImplementation'] = 'Pointwise'
    
    config['LayerName']['phi2']['ParallelizationFactor'] = 1
    config['LayerName']['phi2']['ReuseFactor'] = 8
    config['LayerName']['phi2']['Strategy'] = 'Latency'
    #config['LayerName']['phi2']['ConvImplementation'] = 'Pointwise'
    
    config['LayerName']['phi3']['ParallelizationFactor'] = 1
    config['LayerName']['phi3']['ReuseFactor'] = 8
    config['LayerName']['phi3']['Strategy'] = 'Latency'
    #config['LayerName']['phi3']['ConvImplementation'] = 'Pointwise'

config

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(
    model,
    hls_config=config,
    project_name='hls_deepset_pointwisePhi_ioparallel',
    output_dir='hls_deepset_pointwisePhi_ioparallel',
    part='xcvu13p-flga2577-2-e',
    io_type='io_parallel',
)

hls_model.compile()
hls_model.write()

In [ ]:
Y_hls_pred = hls_model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_hls_pred, axis=1))))

In [ ]:
plt.figure(figsize=(4, 4))
plot_roc(Y_test, Y_hls_pred, ['g','q','w','z','t'])

In [ ]:
del hls_model, config